In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
!nvidia-smi

Mon Jan 30 07:58:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   34C    P0    43W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../data/'
type_weight = {0:1, 1:6, 2:3}
no_files = 5
path = '../../data/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [5]:
import pickle

sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
sess_eval = sessions[0]+sessions[1]
print(len(sess_eval))

356748


In [6]:
# !rm -rf ./data/
!mkdir -p ./data/

In [7]:
files = sorted(
    glob.glob('/otto/data_radek/org/interim/*.parquet')
)
files_split = [glob.glob('/otto/data_radek/org/test.parquet')] + [glob.glob('./data/xgb_train_x.parquet')] + list_in_chunks(files, no_chunks=len(files)//no_files)

In [8]:
len(files_split)

28

In [7]:
!nvidia-smi

Mon Jan 30 07:59:24 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   34C    P0    47W / 163W |    646MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   

In [8]:
import cudf
import numpy as np
import gc
import os

import glob

In [10]:
def get_candidates(df, co, prefix, treshhold=2, treshhold_topn=15):
    df = df.sort_values(["session", "ts"])
    df['d'] = df.groupby('session').ts.diff()
    df.d = (df.d > 60*60*2).astype('int16').fillna(0)
    df.d = df.groupby('session').d.cumsum()
    df = df.merge(
        df.groupby(['session', 'd']).ts.max().reset_index().rename(columns={'ts': 'ts_max_2'}),
        how='left',
        on=['session', 'd']
    )
    df['type_w'] = df['type'].isin([0,1])
    df = df.merge(
        co,
        how='left',
        on=['aid']
    )
    df['wgt'] = df['wgt'].fillna(0)
    df['wgt_1'] = df['wgt_1']*df['wgt']
    df['wgt_2'] = df['wgt_2']*df['wgt']
    df['dummy'] = 1
    df['t_wgt_1'] = df['wgt_1']*df['type']
    df['t_wgt_2'] = df['wgt_2']*df['type']
    df['t_dummy'] = df['dummy']*df['type']
    df['d_wgt_1'] = df['wgt_1']*(df['d']==0)
    df['d_wgt_2'] = df['wgt_2']*(df['d']==0)
    df['d_dummy'] = df['dummy']*(df['d']==0)
    df['l_dummy'] = df['dummy']*(df['ts']==df['ts_max_2'])
    df['l_wgt_1'] = df['wgt_1']*(df['ts']==df['ts_max_2'])
    df['l_wgt_2'] = df['wgt_2']*(df['ts']==df['ts_max_2'])

    agg = {
        'wgt': 'sum',
        'dummy': 'sum',
        'wgt_1': 'sum',
        'wgt_2': 'sum',
        't_dummy': 'sum',
        't_wgt_1': 'sum',
        't_wgt_2': 'sum',
        'd_wgt_1': 'sum',
        'd_wgt_2': 'sum',
        'd_dummy': 'sum',
        'l_dummy': 'sum',
        'l_wgt_1': 'sum',
        'l_wgt_2': 'sum'
    }

    df = df.groupby(['session', 'cand']).agg(agg)
    df.columns = [prefix + '_' + str(x) for x in df.columns]
    df = df.reset_index()
    return(df)

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [11]:
mainprefix = 'chris_'

In [12]:
import pickle

for igfold in range(5):
    print("################# " + str(igfold) + " #####################")
    sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
    if igfold == 0:
        sess_eval = sessions[0]+sessions[1]
    elif igfold == 1:
        sess_eval = sessions[2]+sessions[3]
    elif igfold == 2:
        sess_eval = sessions[4]+sessions[5]
    elif igfold == 3:
        sess_eval = sessions[6]+sessions[7]
    elif igfold == 4:
        sess_eval = sessions[8]+sessions[9]
    print(len(sess_eval))
    files = sorted(glob.glob('./data_folds/fold_' + str(igfold) + '/top_40_aids_v*'))
    for icofile, cofile in enumerate(files):
        print(cofile)
        prefix = mainprefix + '_' + str(icofile) + '_'
        for dataset in ['train', 'sub']:
            for treshhold_topn in [15]:
                print(dataset, treshhold_topn)
                os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn))
                co = cudf.read_parquet(cofile)
                co.columns = ['aid', 'cand', 'wgt']
                if dataset == 'train':
                    df = cudf.read_parquet('./data/xgb_train_x.parquet')
                    df = df[df['session'].isin(sess_eval)]
                else:
                    df = cudf.read_parquet('../../data/test.parquet')
                df = df.merge(
                    df_type,
                    how='left',
                    on='type'
                )
                df['session'] = df['session'].astype('int32')
                df['aid'] = df['aid'].astype('int32')
                df.ts = (df.ts/1000).astype('int32')
                df.drop(['type'], axis=1, inplace=True)
                df = df.rename(columns={'type_': 'type'})
                df = df.merge(
                    df[['session']].groupby(['session']).size().reset_index().rename(columns={0: 'session_len'}),
                    how='left',
                    on='session'
                )
                df = df.merge(
                    df[['session', 'ts']].groupby(['session']).max().reset_index().rename(columns={'ts': 'ts_max'}),
                    how='left',
                    on='session'
                )
                df = df.sort_values(['session', 'ts'], ascending=[True, False])
                df['dummy'] = 1
                df['rank'] = df.groupby(['session']).dummy.cumsum()
                df.drop(['dummy'], axis=1, inplace=True)
                min_val = (2 ** 0.1-1)
                max_val = (2 ** 1-1)
                df['wgt_1'] = (min_val+(max_val-min_val)*(df['rank']-1)/(df['session_len']))
                min_val = (2 ** 0.5-1)
                max_val = (2 ** 1-1)
                df['wgt_2'] = (min_val+(max_val-min_val)*(df['rank']-1)/(df['session_len']))
                if dataset == 'train':
                    session_lists = list_in_chunks(df['session'].drop_duplicates().to_pandas().values.tolist(), no_chunks=20)
                    out = []    
                    for session_list in session_lists:
                        df_tmp = df[df['session'].isin(session_list)]
                        df_tmp = get_candidates(df_tmp, co, prefix, treshhold=2, treshhold_topn=treshhold_topn)
                        out.append(df_tmp)
                    del df
                    del co
                    gc.collect()
                    df = cudf.concat(out)
                    df['session'] = df['session'].astype('int32')
                    df['cand'] = df['cand'].astype('int32')
                    df.to_parquet('./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn) + '/cand.parquet')
                    del df
                    gc.collect()
                else:
                    sub_sessions = pickle.load(open('./data/split/sub_sessoins_10.pickle', 'rb'))
                    out = []
                    tmpfile = './data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn) + '/cand.parquet'
                    os.system('mkdir -p ' + '/'.join(tmpfile.replace('./data_folds/fold_' + str(igfold) + '/', './data_folds/fold_' + str(igfold) + '/split/').split('/')[:-1]))
                    for i, session_list in enumerate(sub_sessions):
                        df_tmp = df[df['session'].isin(session_list)]
                        df_tmp = get_candidates(df_tmp, co, prefix, treshhold=2, treshhold_topn=treshhold_topn)
                        gc.collect()
                        df_tmp['session'] = df_tmp['session'].astype('int32')
                        df_tmp['cand'] = df_tmp['cand'].astype('int32')
                        df_tmp.to_parquet(
                            '/'.join(tmpfile.replace('./data_folds/fold_' + str(igfold) + '/', './data_folds/fold_' + str(igfold) + '/split/').split('/')[:-1]) + '/cand_' + str(i) + '.parquet'
                        )
                        del df_tmp
                        gc.collect()



################# 0 #####################
356748
./data_folds/fold_0/top_40_aids_v115.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v217.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v220.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v226.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v232.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v235.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v239.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v700.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v701.parquet
train 15
sub 15
./data_folds/fold_0/top_40_aids_v93.parquet
train 15
sub 15
################# 1 #####################
356748
./data_folds/fold_1/top_40_aids_v115.parquet
train 15
sub 15
./data_folds/fold_1/top_40_aids_v217.parquet
train 15
sub 15
./data_folds/fold_1/top_40_aids_v220.parquet
train 15
sub 15
./data_folds/fold_1/top_40_aids_v226.parquet
train 15
sub 15
./data_folds/fold_1/top_40_aids_v232.parquet
trai

In [17]:
7513567*20

150271340